In [1]:
import sys
import json
import time

IS_PY3 = sys.version_info.major == 3

from urllib.request import urlopen
from urllib.request import Request
from urllib.error import URLError
from urllib.parse import urlencode
timer = time.perf_counter


API_KEY = 'smxkOHWjqLVljEmIry5vuSYI'
SECRET_KEY = 'ky3FZeSSDurxLyLLqZ2kPaEnts9NH1W1'

# 需要识别的文件
AUDIO_FILE = 'speech-demo-master/rest-api-asr/python/audio/16k.pcm'  # 只支持 pcm/wav/amr 格式，极速版额外支持m4a 格式
# 文件格式
FORMAT = AUDIO_FILE[-3:];  # 文件后缀只支持 pcm/wav/amr 格式，极速版额外支持m4a 格式

CUID = '123456PYTHON';
# 采样率
RATE = 16000;  # 固定值

# 普通版

DEV_PID = 1537;  # 1537 表示识别普通话，使用输入法模型。根据文档填写PID，选择语言及识别模型
ASR_URL = 'http://vop.baidu.com/server_api'
SCOPE = 'audio_voice_assistant_get'  # 有此scope表示有asr能力，没有请在网页里勾选，非常旧的应用可能没有

"----------------------------------------------------------------------------"
"""  TOKEN start """
TOKEN_URL = 'http://aip.baidubce.com/oauth/2.0/token'
# 获取access_token
def fetch_token():
    params = {'grant_type': 'client_credentials',
              'client_id': API_KEY,
              'client_secret': SECRET_KEY}
    post_data = urlencode(params)
    if (IS_PY3):
        post_data = post_data.encode('utf-8')
    req = Request(TOKEN_URL, post_data)
    try:
        f = urlopen(req)
        result_str = f.read()
    except URLError as err:
#         print('token http response http code : ' + str(err.code))
        result_str = err.read()
    if (IS_PY3):
        result_str = result_str.decode()

#     print(result_str)
    result = json.loads(result_str)
#     print(result)
    if ('access_token' in result.keys() and 'scope' in result.keys()):
        if SCOPE and (not SCOPE in result['scope'].split(' ')):  # SCOPE = False 忽略检查
            raise DemoError('scope is not correct')
        return result['access_token']
    else:
        raise DemoError('MAYBE API_KEY or SECRET_KEY not correct: access_token or scope not found in token response')


"""  TOKEN end """

# 语音识别接口的调用
"----------------------------------------------------------------------------"
token = fetch_token()
speech_data = []
# 打开目标语音文件
with open(AUDIO_FILE, 'rb') as speech_file:
    speech_data = speech_file.read()
length = len(speech_data)
if length == 0:
    raise DemoError('file %s length read 0 bytes' % AUDIO_FILE)

params = {
    'cuid': CUID,
    'token': token,
    'dev_pid': DEV_PID
}
#测试自训练平台需要打开以下信息
#params = {'cuid': CUID, 'token': token, 'dev_pid': DEV_PID, 'lm_id' : LM_ID}
params_query = urlencode(params);

headers = {
    'Content-Type': 'audio/' + FORMAT + '; rate=' + str(RATE),
    'Content-Length': length
}

url = ASR_URL + "?" + params_query

# print post_data
req = Request(ASR_URL + "?" + params_query, speech_data, headers)
# 异常处理，避免出现红色的bug（提前预判可能出现的异常）
try:
    begin = timer()
    f = urlopen(req)
    result_str = f.read()
#     print("Request time cost %f" % (timer() - begin))
except  URLError as err:
    print('asr http response http code : ' + str(err.code))
    result_str = err.read()

if (IS_PY3):
    result_str = str(result_str, 'utf-8')
print("结果：",result_str)
# 文本转字典
result_str = eval(result_str)
result_整理 = result_str['result'][0].split('。')[0]
result_整理
# with open("result.txt", "w") as of:
#     of.write(result_str)

FileNotFoundError: [Errno 2] No such file or directory: 'speech-demo-master/rest-api-asr/python/audio/16k.pcm'